# Safe RL with multiple cars and pedestrians

In [1]:
rng = MersenneTwister(2);

In [2]:
using AutomotivePOMDPs
using MDPModelChecking

In [13]:
using GridInterpolations, StaticArrays, POMDPs, POMDPToolbox, AutoViz, AutomotiveDrivingModels, Reel
using DiscreteValueIteration, DeepQLearning, DeepRL, LocalApproximationValueIteration
using ProgressMeter, Parameters, JLD

In [4]:
cam = FitToContentCamera(0.)

AutoViz.FitToContentCamera(0.0)

In [18]:
include("masking.jl")
include("util.jl")
include("render_helpers.jl")
include("masked_dqn.jl")

## Scenario

In [6]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =  [VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [7]:
pomdp = UrbanPOMDP(env=env,
                   ego_goal = LaneTag(2, 1),
                   max_cars=2, 
                   max_peds=2, 
                   car_birth=0.3, 
                   ped_birth=0.3, 
                   obstacles=false, # no fixed obstacles
                   lidar=false,
                   pos_obs_noise = 0., # fully observable
                   vel_obs_noise = 0.);

## Build and load policy

In [ ]:
mdp = PedCarMDP(env=env, pos_res=2.0, vel_res=2.);
vi_data = JLD.load("pc_util_processed.jld")
vi_policy = ValueIterationPolicy(mdp, vi_data["qmat"], vi_data["util"], vi_data["pol"])
threshold = 0.99
mask = SafetyMask(mdp, vi_policy, threshold);

problem_file="joint-log/log9/problem.jld"
weights_file="joint-log/log9/weights.jld"
solver = jldopen(problem_file, "r") do file
    read(file, "solver")
end
env_ = POMDPEnvironment(pomdp)
graph = TensorFlow.Graph()
train_graph = DeepQLearning.build_graph(solver, env_, graph)
policy = DeepQLearning.restore_policy(env_, solver, train_graph, weights_file)
masked_policy = MaskedDQNPolicy(pomdp, policy, mask)

In [8]:
threshold = 0.9999
ped_mdp = PedMDP(env = env, vel_res=2., pos_res=2., ped_type=VehicleDef(AgentClass.PEDESTRIAN, 1.0, 3.0), ped_birth=0.7)
car_mdp = CarMDP(env = env, vel_res=2., pos_res=2.)
ped_mask_file = "pedmask_new.jld"
car_mask_file = "carmask_new.jld"
ped_mask_data = load(ped_mask_file)
car_mask_data = load(car_mask_file)
ped_mask = SafetyMask(ped_mdp, StormPolicy(ped_mdp, ped_mask_data["risk_vec"], ped_mask_data["risk_mat"]), threshold)
car_mask = SafetyMask(car_mdp, StormPolicy(car_mdp, car_mask_data["risk_vec"], car_mask_data["risk_mat"]), threshold);

In [ ]:
policy = DeepQLearning.restore(problem_file="jointmdp-log/log4/problem.jld", weights_file="jointmdp-log/log4/weights.jld");

In [10]:
include("decomposition.jl")

In [11]:
dec_mask = DecomposedMask(pomdp, car_mask, ped_mask);

In [15]:
rand_pol = RandomMaskedPOMDPPolicy(dec_mask, pomdp, rng);

## Simulation

In [28]:
hr = HistoryRecorder(rng=rng, max_steps=100)
s0 = initialstate(pomdp, rng)
o0 = generate_o(pomdp, s0, rng)
up = FastPreviousObservationUpdater{UrbanObs}()
b0 = initialize_belief(up, o0)
@time hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0);

  0.857617 seconds (6.16 M allocations: 206.902 MiB, 9.24% gc time)


In [29]:
h = hist2
state_hist = h.state_hist
action_hist = h.action_hist
safe_actions_hist = h.ainfo_hist
push!(action_hist, CarMDPAction(NaN))
push!(safe_actions_hist, [CarMDPAction(NaN)])
duration, fps, render_hist = animate_states(pomdp, state_hist, action_hist, safe_actions_hist)
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpS6LC2h", 0x0000000000000065, 2.0, nothing)

In [43]:
s = deepcopy(state_hist[10])
for (i, veh) in enumerate(s)
    println(i, " ", veh.id)
end
sorted_vehicles =  sort(s.entities[1:s.n], by=x->x.id)
for (i, veh) in enumerate(sorted_vehicles)
    println(i, " ", veh.id)
end

1 1
2 101
3 2
4 3
5 102
1 1
2 2
3 3
4 101
5 102


In [41]:
s.entities[2]

Vehicle(101, VehicleState(VecSE2({-5.707, 5.612}, 4.712), Frenet(RoadIndex({1, 0.900857}, {18, 1}), 12.612, -0.293, 3.142), 0.000), VehicleDef(PEDESTRIAN, 1.000, 1.000))

In [23]:
findfirst(s, 1)

3

## Evaluation

In [120]:
@time rewards_mask, steps_mask, violations_mask = evaluation_loop(pomdp, rand_pol, n_ep=100, max_steps=100, rng=rng);
print_summary(rewards_mask, steps_mask, violations_mask)

 63.830275 seconds (541.40 M allocations: 17.082 GiB, 17.15% gc time)
Summary for 100 episodes: 
Average reward: 0.000 
Average # of steps: 100.000 
Average # of violations: 0.000 


In [90]:
@time for ep=1:1000
    hr = HistoryRecorder(rng=rng, max_steps=100)
    s0 = initialstate(pomdp, rng)
    o0 = generate_o(pomdp, s0, rng)
    up = FastPreviousObservationUpdater{UrbanObs}()
    b0 = initialize_belief(up, o0)
    hist2 = simulate(hr, pomdp, rand_pol, up, b0, s0)
    if sum(hist2.reward_hist .< 0.) != 0.
        println("Crash")
        break
    end
end

Crash
  3.233593 seconds (7.35 M allocations: 237.220 MiB, 4.31% gc time)
